In [1]:
import os

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import pickle

In [2]:
dataset = pd.read_csv('course_final_with_keyword.csv')

In [3]:
dataset.shape

(20475, 18)

In [4]:
pkl_file = 'course_data.pickle'
dataset.to_pickle(pkl_file)

In [5]:
dataset_db = pd.read_pickle(pkl_file)
dataset_header = ['index']+ list(dataset_db.columns)
dataset_db['index'] = dataset_db.index + 1
dataset_db = dataset_db[dataset_header]
course_header = ['courseID',
                 'title',
                 'url',
                 'categories',
                 'description_short',
                 'description_long',
                 'difficulty',
                 'duration',
                 'free_option',
                 'number_of_enroll',
                 'rating',
                 'paid_option',
                 'language',
                 'subtitle',
                 'platform',
                 'provider',
                 'image_url',
                 'popularity_index',
                 'keyword']
dataset_db.columns = course_header
dataset_db.head()

,courseID,title,url,categories,description_short,description_long,difficulty,duration,free_option,number_of_enroll,rating,paid_option,language,subtitle,platform,provider,image_url,popularity_index,keyword
0,1,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",Description not availabel,Description not availabel,0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, aws, mainframe, modernization,..."
1,2,Cloud for CEOs,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",This course provides CEOs and presidents a hig...,This course provides CEOs and presidents a hig...,0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"cloud, ceo, course, provide, ceo, president, h..."
2,3,Getting Started with AWS Mainframe Modernizati...,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",The AWS Mainframe Modernization service helps ...,The AWS Mainframe Modernization service helps ...,0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, aws, mainframe, modernization,..."
3,4,Introduction to Robotics on AWS,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing",The robotics industry is growing at a rapid ra...,The robotics industry is growing at a rapid ra...,0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"introduction, robotic, aw, robotic, industry, ..."
4,5,Getting Started with Bottlerocket,https://explore.skillbuilder.aws/learn/course/...,"Computer Science , Cloud , AWS ,Cloud Computing","Bottlerocket is a Linux-based, open-source ope...","Bottlerocket is a Linux-based, open-source ope...",0,0,1,0.0,5.0,0,English,English,0,AWS Skill Builder,https://ccweb.imgix.net/https%3A%2F%2Fwww.clas...,0.415333,"getting, start, bottlerocket, bottlerocket, li..."


In [6]:
# SQL QUERY - CREATE TABLE

course_create_table = ''' CREATE TABLE IF NOT EXISTS course
                        (
                        courseID INTEGER NOT NULL PRIMARY KEY,
                        title TEXT NOT NULL,
                        url TEXT NOT NULL,
                        categories TEXT,
                        description_short TEXT,
                        description_long TEXT,
                        difficulty INTEGER NOT NULL,
                        duration INTEGER NOT NULL,
                        free_option INTEGER,
                        number_of_enroll INTEGER,
                        rating REAL,
                        paid_option TEXT,
                        language TEXT,
                        subtitle TEXT,
                        platform INTEGER NOT NULL,
                        provider TEXT,
                        image_url TEXT,
                        popularity_index REAL
                        );
                        '''


user_create_table = ''' CREATE TABLE IF NOT EXISTS user
                        (
                        userID INTEGER NOT NULL PRIMARY KEY,
                        name varchar(255) NOT NULL,
                        username varchar(255) NOT NULL UNIQUE,
                        password varchar(255) NOT NULL
                        );
                        '''

favourite_create_table = ''' CREATE TABLE IF NOT EXISTS favourite
                             (
                             userID INTEGER NOT NULL,
                             courseID INTEGER NOT NULL,
                             FOREIGN KEY (userID) REFERENCES user (userID),
                             FOREIGN KEY (courseID) REFERENCES course (courseID)
                             );
                             '''

query_create_table = ''' CREATE TABLE IF NOT EXISTS query
                         (
                         queryID INTEGER PRIMARY KEY,
                         query_count INTEGER NOT NULL,
                         userID INTEGER NOT NULL,
                         query_time DATETIME DEFAULT (CURRENT_TIMESTAMP),
                         query_text TEXT,
                         query_duration INTEGER,
                         query_difficulty INTEGER,
                         query_free_option INTEGER,
                         FOREIGN KEY (userID) REFERENCES user (userID)
                         );
                         '''

recom_create_table = ''' CREATE TABLE IF NOT EXISTS recommendation
                         (
                         userID INTEGER NOT NULL,
                         query_count INTEGER NOT NULL,
                         ranking INTEGER NOT NULL,
                         courseID INTEGER NOT NULL,
                         FOREIGN KEY (userID) REFERENCES user (userID),
                         FOREIGN KEY (query_count) REFERENCES query (query_count),
                         FOREIGN KEY (courseID) REFERENCES course (courseID)
                         );
                         '''
skill_create_table ='''CREATE TABLE IF NOT EXISTS skill
                        (
                            userID INTEGER NOT NULL,
                            skill TEXT,
                            FOREIGN KEY (userID) REFERENCES user(userID)
                         );
                         '''
career_path_create_table ='''CREATE TABLE IF NOT EXISTS career_path
                            (
                                userID INTEGER NOT NULL,
                                career TEXT,
                                FOREIGN KEY (userID) REFERENCES user (userID)
                            );
                            '''


In [7]:
# EXECUTE CREATE TABLE

filename = 'C:/Users/DELL/Desktop/Course_Recommendation/BEProject/SystemCode/instance/mydb.db'

conn = sqlite3.connect(filename)
conn.execute(course_create_table)
conn.execute(user_create_table)
conn.execute(favourite_create_table)
conn.execute(query_create_table)
conn.execute(recom_create_table)
conn.execute(skill_create_table)
conn.execute(career_path_create_table)

conn.close()

In [8]:
# SQL QUERY - INSERT DATA

insert_course = ''' INSERT OR REPLACE INTO course
                        (
                        courseID,
                        title,
                        url,
                        categories,
                        description_short,
                        description_long,
                        difficulty,
                        duration,
                        free_option,
                        number_of_enroll,
                        rating,
                        paid_option,
                        language,
                        subtitle,
                        platform,
                        provider,
                        image_url,
                        popularity_index,
                        keyword
                        )
                        VALUES
                        (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        '''



In [9]:
conn.close()

In [10]:
# EXECUTE INSERT DATA

filename = 'C:/Users/DELL/Desktop/Course_Recommendation/BEProject/SystemCode/instance/mydb.db'

conn = sqlite3.connect(filename)
dataset_db.to_sql(name='course', con=conn, if_exists='append', index=False)
conn.close()

IntegrityError: UNIQUE constraint failed: course.courseID

In [ ]:
# REQUERY FOR CHECKING

filename = 'C:/Users/DELL/Desktop/Course_Recommendation/BEProject/SystemCode/instance/mydb.db'
course_table = 'course'

sqlite_conn = sqlite3.connect(filename)
course_check = pd.read_sql('SELECT * FROM ' + course_table, sqlite_conn)
sqlite_conn.close()

In [ ]:
print((((course_check == dataset_db)==False).sum() - dataset_db.isna().sum()).sum())

0


In [ ]:
course_check.isna().sum()

courseID             0
title                0
url                  0
categories           0
description_short    0
description_long     0
difficulty           0
duration             0
free_option          0
number_of_enroll     0
rating               0
paid_option          0
language             0
subtitle             0
platform             0
provider             0
image_url            0
popularity_index     0
keyword              0
dtype: int64